<a href="https://colab.research.google.com/github/au1206/tensorflow_dev_cert/blob/main/00_tensorflow_fundamentals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tensorflow Fundamentals

## Intro to Tensors

In [ ]:
import tensorflow as tf
print(tf.__version__)

2.5.0


### Creating *Tensors*

#### tf.constant()
`def constant(value, dtype=None, shape=None, name='Const')`

Creates a constant tensor from a tensor-like object.

Note: All eager tf.Tensor values are **immutable** (in contrast to
tf.Variable). There is nothing especially _constant_ about the value
returned from tf.constant. This function is not fundamentally different from
tf.convert_to_tensor. The name tf.constant comes from the value being
embedded in a Const node in the tf.Graph. tf.constant is useful
for asserting that the value can be embedded that way.

**If the argument dtype is not specified, then the type is inferred from
the type of value.**

In [ ]:
# Create tensors using tf.constant()
scalar = tf.constant(42)
scalar

<tf.Tensor: shape=(), dtype=int32, numpy=42>

output:  `<tf.Tensor: shape=(), dtype=int32, numpy=42>`

- shape is empty as its just a scalar value
- dtype is int32 can be changed to floats etc, inferred from type of value if not mentioned
- numpy=42, represents the numpy value stored in the tensor

In [ ]:
# Check number of dimensions in tensor
scalar.ndim

0

In [ ]:
# Create a Vector
vector = tf.constant([10,11])
vector

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([10, 11], dtype=int32)>

In [ ]:
# Check dimensions of vector
vector.ndim

1

In [ ]:
# Create a matrix
matrix = tf.constant([[10.,11.],
                      [11.,12.]], dtype=tf.float16) # using float16 dtype for 16-bit precision
matrix

<tf.Tensor: shape=(2, 2), dtype=float16, numpy=
array([[10., 11.],
       [11., 12.]], dtype=float16)>

In [ ]:
matrix.ndim

2

In [ ]:
# Create a higher order tensor

tensor = tf.constant([[[2.,3],[4,5]],
                      [[6,7],[8,9]],
                      [[10,11],[12,13]]])
tensor


<tf.Tensor: shape=(3, 2, 2), dtype=float32, numpy=
array([[[ 2.,  3.],
        [ 4.,  5.]],

       [[ 6.,  7.],
        [ 8.,  9.]],

       [[10., 11.],
        [12., 13.]]], dtype=float32)>

In [ ]:
tensor.ndim

3

#### tf.variable()

A tf.Variable represents a tensor whose value can be changed by running ops on it. Specific ops allow you to read and modify the values of this tensor. Higher level libraries like tf.keras use tf.Variable to store model parameters.

- tf.variables are mutable
- you can reassign the tensor using tf.Variable.assign. Calling assign does not (usually) allocate a new tensor; instead, the existing tensor's memory is reused.
- Creating new variables from existing variables duplicates the backing tensors. Two variables will not share the same memory.
- Variables can also be named which can help you track and debug them. eg `a = tf.Variable(my_tensor, name="T1")`
- Variable names are preserved when saving and loading models. 
- You can turn off gradients for a variable by setting trainable to false at creation. eg `step_counter = tf.Variable(1, trainable=False)`


In [ ]:
a = tf.Variable([7,10])
a

<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([ 7, 10], dtype=int32)>

In [ ]:
a.assign(a+1)

<tf.Variable 'UnreadVariable' shape=(2,) dtype=int32, numpy=array([ 8, 11], dtype=int32)>

#### Random tensors

In [ ]:
# creating random tensors with a specified seed

rand1 = tf.random.Generator.from_seed(42)
rand1

In [ ]:
# sampling from a normal distribution
a = rand1.normal(shape=[3,2])
a

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[-0.7565803 , -0.06854702],
       [ 0.07595026, -1.2573844 ],
       [-0.23193763, -1.8107855 ]], dtype=float32)>

#### tf.random.shuffle()
`def random_shuffle(value, seed=None, name=None)`

Randomly shuffles a tensor along its first dimension.

The tensor is shuffled along dimension 0, such that each value[j] is mapped
to one and only one output[i].

Args:
  - value: A Tensor to be shuffled.
  - seed: A Python integer. Used to create a random seed for the distribution.
  - name: A name for the operation (optional).

Returns:
  - A tensor of same shape and type as value, shuffled along its first
dimension.


Operations that rely on a random seed actually derive it from two seeds: 
- **global seed**
- **operation-level** seed 

`tf.random.set_seed` sets the global seed.

Its interactions with operation-level seeds is as follows:

- If neither the global seed nor the operation seed is set: A randomly picked seed is used for this op.
- If the global seed is set, but the operation seed is not: The system deterministically picks an operation seed in conjunction with the global seed so that it gets a unique random sequence. Within the same version of tensorflow and user code, this sequence is deterministic. However across different versions, this sequence might change. 
- If the operation seed is set, but the global seed is not set: A default global seed and the specified operation seed are used to determine the random sequence.
- If both the global and the operation seed are set: Both seeds are used in conjunction to determine the random sequence.

**Note:** If the code depends on particular seeds to work, specify both global and operation-level seeds explicitly.

https://www.tensorflow.org/api_docs/python/tf/random/set_seed

In [ ]:
t1 = tf.constant([[2,4],[4,2],[3,6]])

In [ ]:
# Shuffling the tensor
tf.random.shuffle(t1,seed=42)

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[3, 6],
       [4, 2],
       [2, 4]], dtype=int32)>

In [ ]:
# A different tensor every time we shuffle (along the first axis(rows))
# the seed here is operation level seed, the results on running again would still be different but across sessions would be similar

tf.random.shuffle(t1, seed=42)

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[3, 6],
       [4, 2],
       [2, 4]], dtype=int32)>

In [ ]:
tf.random.set_seed(42)
tf.random.shuffle(t1, seed=42)

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[2, 4],
       [4, 2],
       [3, 6]], dtype=int32)>

In [ ]:
# now in the same session the global seed is set to 42. so we will get same results no matter how many times we run this 
tf.random.set_seed(42)
tf.random.shuffle(t1, seed=42)

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[2, 4],
       [4, 2],
       [3, 6]], dtype=int32)>

#### Generating tensors from numpy
The main difference between Tensorflow and Numpy is that the tensors are optimised for GPUs.

In [ ]:
import numpy as np

In [ ]:
tf.ones([5,5])

<tf.Tensor: shape=(5, 5), dtype=float32, numpy=
array([[1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.]], dtype=float32)>

In [ ]:
# similar to numpy
tf.zeros([5,5])

<tf.Tensor: shape=(5, 5), dtype=float32, numpy=
array([[0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]], dtype=float32)>

np.arange

`arange([start,] stop[, step,], dtype=None)`

Return evenly spaced values within a given interval.

Values are generated within the half-open interval ``[start, stop)``
For integer arguments the function is equivalent to the Python built-in
`range` function, but returns an ndarray rather than a list.

**NOTE:** When using a non-integer step, such as 0.1, the results will often not be consistent.  It is better to use `numpy.linspace` for these cases.

Parameters

- start : number, optional
    Start of interval.  The interval includes this value.  The default
    start value is 0.
- stop : number
    End of interval.  The interval does not include this value, except
    in some cases where `step` is not an integer and floating point
    round-off affects the length of `out`.
- step : number, optional
    Spacing between values.  For any output `out`, this is the distance
    between two adjacent values, ``out[i+1] - out[i]``.  The default
    step size is 1.  If `step` is specified as a position argument,
    `start` must also be given.
- dtype : dtype
    The type of the output array.  If `dtype` is not given, infer the data
    type from the other input arguments.

Returns

- arange : ndarray
    Array of evenly spaced values.
    For floating point arguments, the length of the result is
    `ceil((stop - start)/step)`.  Because of floating point overflow,
    this rule may result in the last element of `out` being greater
    than `stop`.

In [ ]:
np_A = np.arange(1,25, dtype=np.int32)
np_A

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24], dtype=int32)

In [ ]:
# for converting into a tensor, just pass the numpy array, change the shape, make sure shape is correctly given(same number of elements)
A = tf.constant(np_A, shape=(2,3,4))
A

<tf.Tensor: shape=(2, 3, 4), dtype=int32, numpy=
array([[[ 1,  2,  3,  4],
        [ 5,  6,  7,  8],
        [ 9, 10, 11, 12]],

       [[13, 14, 15, 16],
        [17, 18, 19, 20],
        [21, 22, 23, 24]]], dtype=int32)>

### Getting Info from the Tensor
- Shape
- Rank
- Axis or dimension
- Size

In [ ]:
# creating a rank 4 tensor
rank_4_tensor = tf.zeros(shape=[2,3,4,5])

In [ ]:
rank_4_tensor

<tf.Tensor: shape=(2, 3, 4, 5), dtype=float32, numpy=
array([[[[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]],


       [[[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]]], dtype=float32)>

In [ ]:
rank_4_tensor[0]

<tf.Tensor: shape=(3, 4, 5), dtype=float32, numpy=
array([[[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]]], dtype=float32)>

In [ ]:
rank_4_tensor.shape, rank_4_tensor.ndim, tf.size(rank_4_tensor)

(TensorShape([2, 3, 4, 5]), 4, <tf.Tensor: shape=(), dtype=int32, numpy=120>)

In [ ]:
print("DataType of every element: ", rank_4_tensor.dtype)
print("Number of dimensions(rank): ", rank_4_tensor.ndim)
print("Shape of tensor: ",rank_4_tensor.shape)
print("Elements along 0 axis: ", rank_4_tensor.shape[0])
print("Elements along last axis: ", rank_4_tensor.shape[-1])
print("Total number of elements in the tensor", tf.size(rank_4_tensor).numpy())

DataType of every element:  <dtype: 'float32'>
Number of dimensions(rank):  4
Shape of tensor:  (2, 3, 4, 5)
Elements along 0 axis:  2
Elements along last axis:  5
Total number of elements in the tensor 120


### Indexing Tensors
tensors can be indexed like python lists.

In [ ]:
# first 2 in each dimension
rank_4_tensor[:2,:2,:2,:2]

<tf.Tensor: shape=(2, 2, 2, 2), dtype=float32, numpy=
array([[[[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]]],


       [[[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]]]], dtype=float32)>

In [ ]:
# get first element from each dimension, for each index except final one
rank_4_tensor[:1,:1,:1]

<tf.Tensor: shape=(1, 1, 1, 5), dtype=float32, numpy=array([[[[0., 0., 0., 0., 0.]]]], dtype=float32)>